In [424]:
# 03_log_enrich.ipynb
# Enrich the TED event log

In [425]:
# Force to reload extrernal modules every new cell execution
%reload_ext autoreload
%autoreload 2

In [426]:
### IMPORT ###
from pathlib import Path
import csv
from datetime import datetime
import pandas as pd

In [427]:
### LOCAL IMPORT ###
from config import config_reader
from utilities import read_csv_data_to_df

In [428]:
### GLOBALS ###
yaml_config = config_reader.config_read_yaml("config.yml", "config")
# print(yaml_config) # debug
data_dir = str(yaml_config["DATA_DIR"])
csv_sep = str(yaml_config["CSV_SEP"])
stats_dir = str(yaml_config["STATS_DIR"])
stats_file_valid_bids = str(yaml_config["STATS_FILE_VALID_BIDS"])
event_log_dir = str(yaml_config["EVENT_LOG_DIR"])
event_log_file = str(yaml_config["FILE_EVENT_LOG"]) # main event log file (with LANG to choose)
event_log_text_date = str(yaml_config["FILE_BID_TEXT_DATE"])  # text dates (with LANG to choose)
year_start = int(yaml_config["YEAR_START"])
year_end = int(yaml_config["YEAR_END"])

# INPUT
dic_lan = {"DE":0, "ES":0, "FR":1, "IT":0, "PT":0} # <-- INPUT: set 1 for the desired language; set only one language at time
model_name = "OAI" # <-- INPUT [OAI, HFC]
dic_log_type = {"case_id":"object", "event":"object", "timestamp":"object", "t_type":"object", "amount":"float", "electronic":"object", "framework_agr":"object", "nuts":"object", "country":"object", "cpv_division":"object", "cpv":"object", "case_len":"int"}
dic_dates_type = {"file_name":object, "case_id":object, "text":object, "text_date":object, "text_date_llm":object}
event_new = str(yaml_config["BID_OPENING_EVENT"])

# OUTPUT
log_suffix = "enr"

## FUNCTIONS

## MAIN

In [429]:
### MAIN ###
print()
print("*** PROGRAM START ***")
print()

start_time = datetime.now().replace(microsecond=0)
print("Start process:", str(start_time))
print()


*** PROGRAM START ***

Start process: 2024-09-12 11:26:06



In [430]:
print(">> Settings")
lang_code = None
key_with_value_1 = [key for key, value in dic_lan.items() if value == 1]
if key_with_value_1: 
    lang_code = key_with_value_1[0]
print("Desired language code for LLM:", lang_code)

# INPUT
# Initial event log
event_log_file_lang = event_log_file.replace("LANG", lang_code).replace("YS", str(year_start)).replace("YE", str(year_end))
path_log_file_lang = Path(event_log_dir) / event_log_file_lang
print("File input:", path_log_file_lang)

# Dates extracted
event_log_text_date_lang = event_log_text_date.replace("LANG", lang_code)
path_log_text_date_lang = Path(data_dir) / f"{Path(event_log_text_date_lang).stem}_{model_name}.csv"
print("File input with new dates:", path_log_text_date_lang)

# OUTPUT
# Event log enriched
event_log_file_lang_enr = f"{Path(event_log_file_lang).stem}_{log_suffix}.csv"
path_log_file_lang_enr = Path(data_dir) / event_log_file_lang_enr
print("File output:", path_log_file_lang_enr)

out_dir = Path(stats_dir)
out_dir.mkdir(exist_ok=True)

out_dir = Path(data_dir)
out_dir.mkdir(exist_ok=True)

>> Settings
Desired language code for LLM: ES
File input: /Volumes/SAMSUNG-PHD/PhD/Corsi da seguire/Knowledge management and information extraction from structured and unstructured data for process mining (techniques, algorithms, and tools)/Python - GUUE in CSV per LOG/data_log/TED_log_2016-2022_ES.csv
File input with new dates: data/bid_opening_text_ES_date_llm_OAI.csv
File output: data/TED_log_2016-2022_ES_enr.csv


In [431]:
# Reading CSV file text and dates to be extracted by LLM
print(">> Reading CSV input file (log not enriched)")
print("Path:", str(path_log_file_lang))
df_event_log = read_csv_data_to_df(path_log_file_lang, dic_log_type, csv_sep) # initiale vent log (not enriched)
df_event_log_len = len(df_event_log)
print("Rows in dataframe:", df_event_log_len)
print("Columns in dataframe:", df_event_log.columns)
print("Cases in event log (not enriched):", df_event_log["case_id"].nunique())

>> Reading CSV input file (log not enriched)
Path: /Volumes/SAMSUNG-PHD/PhD/Corsi da seguire/Knowledge management and information extraction from structured and unstructured data for process mining (techniques, algorithms, and tools)/Python - GUUE in CSV per LOG/data_log/TED_log_2016-2022_ES.csv
Reading CSV with input col_type: {'case_id': 'object', 'event': 'object', 'timestamp': 'object', 't_type': 'object', 'amount': 'float', 'electronic': 'object', 'framework_agr': 'object', 'nuts': 'object', 'country': 'object', 'cpv_division': 'object', 'cpv': 'object', 'case_len': 'int'}
Rows in dataframe: 109343
Columns in dataframe: Index(['case_id', 'event', 'timestamp', 't_type', 'amount', 'electronic',
       'framework_agr', 'nuts', 'country', 'cpv_division', 'cpv', 'case_len'],
      dtype='object')
Cases in event log (not enriched): 33033


In [432]:
df_event_log.head(5)

,case_id,event,timestamp,t_type,amount,electronic,framework_agr,nuts,country,cpv_division,cpv,case_len
0,2016100531,PUBLICATION,2016-03-23,S,768600.00,N,N,ES521,ES,50,50420000,3
1,2016100531,PARTICIPATION,2016-05-02,S,768600.00,N,N,ES521,ES,50,50420000,3
2,2016100531,AWARD,2016-07-22,S,768600.00,N,N,ES521,ES,50,50420000,3
3,2016100696,PUBLICATION,2016-03-23,W,4847774.52,N,N,NaN,ES,45,45213312,3
4,2016100696,PARTICIPATION,2016-05-03,W,4847774.52,N,N,NaN,ES,45,45213312,3


In [433]:
# Count the number of valid dates (other than 0) 
print(">> Reading CSV text extracted dates file")
print("Path:", str(path_log_file_lang))
df_bid_text = read_csv_data_to_df(path_log_text_date_lang, dic_dates_type, csv_sep)
df_bid_text = df_bid_text[df_bid_text['case_id'].isin(df_event_log['case_id'])]
df_bid_text_len = len(df_bid_text)
df_bid_text_valid = df_bid_text[df_bid_text["text_date"]!="0"]
df_bid_text_valid_len = len(df_bid_text_valid)
print(f"Number of not empty dates ({lang_code}): {df_bid_text_valid_len} / {df_bid_text_len}")
print("Cases in text dates:", df_bid_text["case_id"].nunique())
print("Valid cases in text dates:", df_bid_text_valid["case_id"].nunique())

percent_cases = round((df_bid_text_valid["case_id"].nunique() / df_bid_text["case_id"].nunique()), 2)

df = pd.DataFrame(columns=['country', 'total_cases', 'text_cases', 'percent_cases'])
df_bid_stats = pd.DataFrame({
    'country': [lang_code],
    'total_cases': [df_bid_text["case_id"].nunique()],
    'text_cases': [df_bid_text_valid["case_id"].nunique()],
    'percent_cases': [percent_cases]
})

path_stats = Path(stats_dir) / stats_file_valid_bids
if path_stats.exists():
    df_bid_stats.to_csv(path_stats, sep=csv_sep, index=False, mode="a", header=False)
else:
    df_bid_stats.to_csv(path_stats, sep=csv_sep, index=False, mode="w", header=True)

>> Reading CSV text extracted dates file
Path: /Volumes/SAMSUNG-PHD/PhD/Corsi da seguire/Knowledge management and information extraction from structured and unstructured data for process mining (techniques, algorithms, and tools)/Python - GUUE in CSV per LOG/data_log/TED_log_2016-2022_ES.csv
Reading CSV with input col_type: {'file_name': <class 'object'>, 'case_id': <class 'object'>, 'text': <class 'object'>, 'text_date': <class 'object'>, 'text_date_llm': <class 'object'>}
Number of not empty dates (ES): 23038 / 30536
Cases in text dates: 30536
Valid cases in text dates: 23038


In [434]:
df_bid_stats

,country,total_cases,text_cases,percent_cases
0,ES,30536,23038,0.75


In [435]:
df_bid_text_valid.head(5)

,file_name,case_id,text,text_date,text_date_llm
0,2016-OJS001-00000032-es-ts.pdf,201632,Fecha: 18/02/2016 | Hora local: 12:00 | - - Su...,Fecha: 18/02/2016,2016-02-18
1,2016-OJS001-00000036-es-ts.pdf,201636,Fecha: 17/02/2016 | Hora local: 12:00 | Lugar:...,Fecha: 17/02/2016,2016-02-17
16,2016-OJS002-00001140-es-ts.pdf,20161140,Fecha: 17/02/2016 | Hora local: 08:00 | Lugar:...,Fecha: 17/02/2016,2016-02-17
17,2016-OJS003-00002226-es-ts.pdf,20162226,Fecha: 09/03/2016 | Hora local: 12:30 | Lugar:...,Fecha: 09/03/2016,2016-03-09
18,2016-OJS003-00002227-es-ts.pdf,20162227,Fecha: 24/02/2016 | Hora local: 12:30 | Lugar:...,Fecha: 24/02/2016,2016-02-24


In [436]:
# Enriching the event log
print(">> Enriching the event log")

# Step 1: Remove duplicates from df_bid_text_valid to ensure each case_id is unique
df_bid_text_valid_unique = df_bid_text_valid.drop_duplicates(subset=['case_id'])

# Step 2: Find the unique case_ids that are in both dataframes
common_case_ids = df_event_log['case_id'].unique() # get unique case_id
print("Cases in event log not enriched:", len(common_case_ids))

df_common = df_bid_text_valid_unique[df_bid_text_valid_unique['case_id'].isin(common_case_ids)]
df_common_len = len(df_common)
print("Cases in common with LLM extraction to be enriched:", df_common["case_id"].nunique())

>> Enriching the event log
Cases in event log not enriched: 33033
Cases in common with LLM extraction to be enriched: 23038


In [437]:
# Step 3: Create a list to store new rows
new_rows = []

# Step 4: Iterate over each common case_id to create one new row per case_id
i = 1
for idx, row in df_common.iterrows():
    print(f"Row {i} / {df_common_len}")
    case_id = row['case_id']
    text_date_llm = row['text_date_llm']
    
    # Find the first occurrence of the current case_id in df_event_log
    existing_values = df_event_log[df_event_log['case_id'] == case_id].iloc[0]
    
    # Create a new row with the existing values and the date from df_bid_text_valid
    new_row = {
        'case_id': case_id,
        'event': 'BID-OPENING',
        'timestamp': text_date_llm,
        't_type': existing_values['t_type'],
        'amount': existing_values['amount'],
        'electronic': existing_values['electronic'],
        'framework_agr': existing_values['framework_agr'],
        'nuts': existing_values['nuts'],
        'country': existing_values['country'],
        'cpv_division': existing_values['cpv_division'],
        'cpv': existing_values['cpv'],
        'case_len': existing_values['case_len']
    }
    
    # Add the new row to the list
    new_rows.append(new_row)
    i+=1

Row 1 / 23038
Row 2 / 23038
Row 3 / 23038
Row 4 / 23038
Row 5 / 23038
Row 6 / 23038
Row 7 / 23038
Row 8 / 23038
Row 9 / 23038
Row 10 / 23038
Row 11 / 23038
Row 12 / 23038
Row 13 / 23038
Row 14 / 23038
Row 15 / 23038
Row 16 / 23038
Row 17 / 23038
Row 18 / 23038
Row 19 / 23038
Row 20 / 23038
Row 21 / 23038
Row 22 / 23038
Row 23 / 23038
Row 24 / 23038
Row 25 / 23038
Row 26 / 23038
Row 27 / 23038
Row 28 / 23038
Row 29 / 23038
Row 30 / 23038
Row 31 / 23038
Row 32 / 23038
Row 33 / 23038
Row 34 / 23038
Row 35 / 23038
Row 36 / 23038
Row 37 / 23038
Row 38 / 23038
Row 39 / 23038
Row 40 / 23038
Row 41 / 23038
Row 42 / 23038
Row 43 / 23038
Row 44 / 23038
Row 45 / 23038
Row 46 / 23038
Row 47 / 23038
Row 48 / 23038
Row 49 / 23038
Row 50 / 23038
Row 51 / 23038
Row 52 / 23038
Row 53 / 23038
Row 54 / 23038
Row 55 / 23038
Row 56 / 23038
Row 57 / 23038
Row 58 / 23038
Row 59 / 23038
Row 60 / 23038
Row 61 / 23038
Row 62 / 23038
Row 63 / 23038
Row 64 / 23038
Row 65 / 23038
Row 66 / 23038
Row 67 / 23038
Row 

In [438]:
print("New rows:", len(new_rows))

New rows: 23038


In [439]:
# Step 5: Convert the list of new rows into a DataFrame
new_rows_df = pd.DataFrame(new_rows)

# Step 6: Append the new rows to the df_event_log_enriched dataframe
df_event_log_enriched = pd.concat([df_event_log, new_rows_df], ignore_index=True)
print("Cases in event log enriched:", df_event_log_enriched["case_id"].nunique())

# Step 7: Drop any duplicates after the new rows have been added
# df_event_log.drop_duplicates(subset=['case_id', 'event', 'timestamp'], inplace=True)
df_event_log_enriched.drop_duplicates(inplace=True)
print("Cases in event log enriched (after drop duplicates):", df_event_log_enriched["case_id"].nunique())

# Step 8: Sort the dataframe by 'case_id' and 'timestamp'
df_event_log_enriched.sort_values(by=['case_id', 'timestamp'], inplace=True)

# Step 9: Reset the index of the dataframe after sorting
df_event_log_enriched.reset_index(drop=True, inplace=True)

Cases in event log enriched: 33033
Cases in event log enriched (after drop duplicates): 33033


In [440]:
# Check
print(">> Event log enriched checks")
print("Cases in dataframe enriched:", df_event_log_enriched["case_id"].nunique())
print("Rows in dataframe enriched:", len(df_event_log_enriched))
print("Columns in dataframe enriched:", df_event_log_enriched.columns)

>> Event log enriched checks
Cases in dataframe enriched: 33033
Rows in dataframe enriched: 132381
Columns in dataframe enriched: Index(['case_id', 'event', 'timestamp', 't_type', 'amount', 'electronic',
       'framework_agr', 'nuts', 'country', 'cpv_division', 'cpv', 'case_len'],
      dtype='object')


In [441]:
df_event_log_enriched.head(5)

,case_id,event,timestamp,t_type,amount,electronic,framework_agr,nuts,country,cpv_division,cpv,case_len
0,2016100531,PUBLICATION,2016-03-23,S,768600.00,N,N,ES521,ES,50,50420000,3
1,2016100531,PARTICIPATION,2016-05-02,S,768600.00,N,N,ES521,ES,50,50420000,3
2,2016100531,BID-OPENING,2016-05-17,S,768600.00,N,N,ES521,ES,50,50420000,3
3,2016100531,AWARD,2016-07-22,S,768600.00,N,N,ES521,ES,50,50420000,3
4,2016100696,PUBLICATION,2016-03-23,W,4847774.52,N,N,NaN,ES,45,45213312,3


In [442]:
df_event_log_enriched.dtypes

case_id           object
event             object
timestamp         object
t_type            object
amount           float64
electronic        object
framework_agr     object
nuts              object
country           object
cpv_division      object
cpv               object
case_len           int64
dtype: object

In [443]:
df_event_log_enriched.shape

(132381, 12)

>> Saving event log enriched
Path: data/TED_log_2016-2022_ES_enr.csv



In [415]:
print(">> Count distinct values of timestamp (enriched)")
event_time_dist = df_event_log_enriched['timestamp'].value_counts().reset_index()

>> Count distinct values of timestamp (enriched)


In [445]:
# Check for missing timestamp (0)
event_time_dist

,timestamp,count
0,0,66
1,2020-11-16,21
2,2019-08-05,17
3,2021-06-30,17
4,2020-11-13,17
...,...,...
2064,2016-10-11,1
2065,2017-11-29,1
2066,2016-11-28,1
2067,2017-10-03,1


In [446]:
df_event_log_enriched_zero = df_event_log_enriched[df_event_log_enriched["timestamp"] == "0"]

In [447]:
df_event_log_enriched_zero

,case_id,event,timestamp,t_type,amount,electronic,framework_agr,nuts,country,cpv_division,cpv,case_len
560,2016130367,BID-OPENING,0,U,785525.67,N,Y,ES511,ES,30,30197630,3
2370,2016200901,BID-OPENING,0,S,10752205.60,N,N,ES21,ES,79,79714000,3
3813,2016249829,BID-OPENING,0,U,3820533.46,N,N,ES130,ES,39,39518000,3
5049,2016280386,BID-OPENING,0,S,223140.50,N,N,ES220,ES,71,71410000,5
5231,2016284716,BID-OPENING,0,S,478098.10,N,N,ES511,ES,85,85311300,3
...,...,...,...,...,...,...,...,...,...,...,...,...
85812,202080235,BID-OPENING,0,S,226270.00,N,N,ES130,ES,50,50324100,3
85863,202080554,BID-OPENING,0,S,540000.00,N,N,ES120,ES,50,50750000,3
86062,202085319,BID-OPENING,0,S,1157575.52,N,N,ES615,ES,50,50532200,2
86354,202094920,BID-OPENING,0,S,983839.14,N,N,ES618,ES,90,90919200,3


In [448]:
df_event_log_enriched_zero["event"].unique()

array(['BID-OPENING'], dtype=object)

In [449]:
print(" Removing rows with timestamp = 0")
df_event_log_enriched = df_event_log_enriched[df_event_log_enriched['timestamp'] != "0"]

 Removing rows with timestamp = 0


In [450]:
# Check if 0 have been removed
df_event_log_enriched_zero = df_event_log_enriched[df_event_log_enriched["timestamp"] == "0"]
df_event_log_enriched_zero

,case_id,event,timestamp,t_type,amount,electronic,framework_agr,nuts,country,cpv_division,cpv,case_len


In [452]:
print(">> Check the last event")
print("Lang:", lang_code)

# Convert the 'timestamp' column to datetime type if it's not already
# df_event_log_enriched['timestamp'] = pd.to_datetime(df_event_log_enriched['timestamp'])

# Sort the DataFrame by 'case_id' and 'timestamp'
df_event_log_enriched = df_event_log_enriched.sort_values(by=['case_id', 'timestamp'])

# Get the last event for each 'case_id'
last_events = df_event_log_enriched.groupby('case_id').last().reset_index()

# Count how many cases end with each event
event_name_counts = last_events['event'].value_counts().reset_index()

# Rename columns for clarity
event_name_counts.columns = ['event', 'case_count']

# Show the result
event_name_counts

>> Check the last event
Lang: ES


,event,case_count
0,AWARD,27909
1,BID-OPENING,2833
2,CONTRACT-END,1493
3,PARTICIPATION,796
4,CONTRACT-START,2


In [458]:
# Add eventul missing cases
# Cases in initial event log
# Find the Case IDs that are present in df_disco but missing in df_enr_disco
# Extract the unique 'Case ID' values from both datasets

case_ids_enr = set(df_event_log_enriched['case_id'].unique())
case_ids = set(df_event_log['case_id'].unique())

missing_case_ids = case_ids - case_ids_enr

# Filter the rows from df_disco that correspond to the missing 'Case ID's
missing_rows = df_event_log[df_event_log['case_id'].isin(missing_case_ids)]

print("Missing rows:", len(missing_rows))

# Cases in enriched event log

# Concatenate the filtered original dataframe with the missing rows
df_event_log_enriched_final = pd.concat([df_event_log_enriched, missing_rows], ignore_index=True)

# Convert 'Complete Timestamp' to datetime to ensure correct sorting
df_event_log_enriched_final['timestamp'] = pd.to_datetime(df_event_log_enriched_final['timestamp'])

# Sort by 'Case ID' and 'Complete Timestamp'
df_event_log_enriched_final = df_event_log_enriched_final.sort_values(by=['case_id', 'timestamp'])


Missing rows: 0


In [459]:
# Create a file with the texts extracted from the PDFs
print(">> Saving event log enriched")
print("Path:", path_log_file_lang_enr)
df_event_log_enriched_final.to_csv(path_log_file_lang_enr, sep=csv_sep, index=False, quoting=csv.QUOTE_ALL)
print()

>> Saving event log enriched
Path: data/TED_log_2016-2022_ES_enr.csv



In [423]:
# program end
end_time = datetime.now().replace(microsecond=0)
delta_time = end_time - start_time

print()
print("End process:", end_time)
print("Time to finish:", delta_time)
print()

print()
print("*** PROGRAM END ***")
print()


End process: 2024-09-11 18:09:08
Time to finish: 0:00:03


*** PROGRAM END ***

